In [ ]:
### Import Libraries.

library("nichenetr")
library("Seurat")
library("SeuratObject")
library("tidyverse")
library("RColorBrewer")
library("readxl")
library("qs")
library("cowplot")

In [ ]:
### Load Data.

setwd("/folder")
Seurat <- qread("Seurat.qs")
Idents(Seurat) <- "Cluster_Column"

In [ ]:
### Load NicheNet Networks.

lr_network <- readRDS(url("https://zenodo.org/record/7074291/files/lr_network_human_21122021.rds")) %>% distinct(from, to)
ligand_target_matrix <- readRDS(url("https://zenodo.org/record/7074291/files/ligand_target_matrix_nsga2r_final.rds"))
weighted_networks <- readRDS(url("https://zenodo.org/record/7074291/files/weighted_networks_nsga2r_final.rds"))

In [ ]:
### Define Expressed Genes & Potential Ligands.

receiver <- "Cluster_1"
expressed_genes_receiver <- get_expressed_genes(receiver, Seurat, pct = 0.5)
expressed_receptors <- intersect(unique(lr_network$to), expressed_genes_receiver)

In [ ]:
### Define Sender-Agnostic and Sender-Focused Ligands.

potential_ligands <- c("Ligand_1","Ligand_2","Ligand_3","Ligand_4") %>%
                     intersect(colnames(ligand_target_matrix))

sender_celltypes <- c("Cluster_2")
expressed_genes_sender <- sender_celltypes %>% lapply(get_expressed_genes, Seurat, 0.1) %>% unlist() %>% unique()
potential_ligands_focused <- intersect(potential_ligands, expressed_genes_sender)

In [ ]:
### Define Genes of Interest and Background.

genes <- read_xlsx("marker_genes.xlsx")
geneset_oi <- intersect(unique(genes$feature), rownames(ligand_target_matrix))
background_expressed_genes <- intersect(expressed_genes_receiver, rownames(ligand_target_matrix))

In [ ]:
### Ligand Activity Analysis.

ligand_activities <- predict_ligand_activities(
  geneset = geneset_oi,
  background_expressed_genes = background_expressed_genes,
  ligand_target_matrix = ligand_target_matrix,
  potential_ligands = potential_ligands
) %>% arrange(-aupr_corrected) %>% mutate(rank = rank(desc(aupr_corrected)))

best_upstream_ligands <- ligand_activities %>% top_n(30, aupr_corrected) %>% pull(test_ligand)

In [ ]:
### Visualize Ligand Activity.

vis_ligand_aupr <- ligand_activities %>% 
  filter(test_ligand %in% best_upstream_ligands) %>% 
  column_to_rownames("test_ligand") %>% 
  select(aupr_corrected) %>% as.matrix()

make_heatmap_ggplot(vis_ligand_aupr, "Prioritized ligands", "Ligand activity",
                     legend_title = "AUPR", color = "darkorange")

In [ ]:
### Infer Target Genes and Receptors.

active_ligand_target_links_df <- best_upstream_ligands %>% 
  lapply(get_weighted_ligand_target_links, geneset = geneset_oi,
         ligand_target_matrix = ligand_target_matrix, n = 50) %>%
  bind_rows() %>% drop_na()

active_ligand_target_links <- prepare_ligand_target_visualization(active_ligand_target_links_df,
                                                                  ligand_target_matrix, cutoff = 0.33)

order_ligands <- rev(intersect(best_upstream_ligands, colnames(active_ligand_target_links)))
order_targets <- intersect(active_ligand_target_links_df$target, rownames(active_ligand_target_links))
vis_ligand_target <- t(active_ligand_target_links[order_targets, order_ligands])

make_heatmap_ggplot(vis_ligand_target, "Prioritized ligands", "Predicted target genes",
                     color = "purple", legend_title = "Regulatory potential") +
  scale_fill_gradient2(low = "whitesmoke", high = "purple")

In [ ]:
### Receptor Links.

ligand_receptor_links_df <- get_weighted_ligand_receptor_links(best_upstream_ligands, expressed_receptors,
                                                               lr_network, weighted_networks$lr_sig)
vis_ligand_receptor_network <- prepare_ligand_receptor_visualization(ligand_receptor_links_df, best_upstream_ligands,
                                                                     order_hclust = "both")
make_heatmap_ggplot(t(vis_ligand_receptor_network), y_name = "Ligands", x_name = "Receptors",
                    color = "mediumvioletred", legend_title = "Prior interaction potential")

In [ ]:
### Sender-focused DotPlot.

DotPlot(Seurat, features = rev(best_upstream_ligands), cols = "RdYlBu") +
  coord_flip() +
  scale_size(range = c(0, 10))

In [ ]:
### LFC in Sender Cells.

celltype_order <- levels(Idents(Seurat))
DE_table_top_ligands <- lapply(
  celltype_order[celltype_order %in% sender_celltypes],
  get_lfc_celltype,
  seurat_obj = SeuratObject,
  condition_colname = "Cluster_Column",
  condition_oi = condition_oi,
  condition_reference = condition_reference,
  celltype_col = "Cluster_Column",
  min.pct = 0, logfc.threshold = 0,
  features = best_upstream_ligands
)
DE_table_top_ligands <- reduce(DE_table_top_ligands, full_join) %>% column_to_rownames("gene")
vis_ligand_lfc <- as.matrix(DE_table_top_ligands[rev(best_upstream_ligands), , drop = FALSE])
p_lfc <- make_threecolor_heatmap_ggplot(vis_ligand_lfc,
                                        "Prioritized ligands", "LFC in Sender",
                                        low_color = "midnightblue", mid_color = "white",
                                        mid = median(vis_ligand_lfc), high_color = "red",
                                        legend_title = "LFC")

In [ ]:
### Combine Plots.

figures_without_legend <- cowplot::plot_grid(
  p_ligand_aupr + theme(legend.position = "none"),
  p_dotplot + theme(legend.position = "none", axis.ticks = element_blank(),
                    axis.title.y = element_blank(),
                    axis.title.x = element_text(size = 12),
                    axis.text.y = element_text(size = 9),
                    axis.text.x = element_text(size = 9,  angle = 90, hjust = 0)) + ylab("Expression in Sender"),
  p_lfc + theme(legend.position = "none", axis.title.y = element_blank()),
  p_ligand_target + theme(legend.position = "none", axis.title.y = element_blank()),
  align = "hv", nrow = 1,
  rel_widths = c(ncol(vis_ligand_aupr)+6, ncol(vis_ligand_lfc)+7, ncol(vis_ligand_lfc)+8, ncol(vis_ligand_target))
)

legends <- cowplot::plot_grid(
  ggpubr::as_ggplot(ggpubr::get_legend(p_ligand_aupr)),
  ggpubr::as_ggplot(ggpubr::get_legend(p_dotplot)),
  ggpubr::as_ggplot(ggpubr::get_legend(p_ligand_target)),
  nrow = 1, align = "h", rel_widths = c(1.5,1,1,1)
)

combined_plot <- cowplot::plot_grid(figures_without_legend, legends, rel_heights = c(10,5), nrow = 2, align = "hv")
combined_plot